# Análise comparativa

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

## 1. Obtenção dos dados:

In [2]:
df = pd.read_csv("../data/raw/day.csv")
df['dteday'] = pd.to_datetime(df['dteday'])
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,instant,Índice de registro,quantitativa,discreta
1,dteday,Data,qualitativa,ordinal
2,season,Estação do ano,qualitativa,ordinal
3,yr,Ano,qualitativa,ordinal
4,mnth,Mês,qualitativa,ordinal
5,holiday,Se o dia é feriado ou não,qualitativa,nominal
6,weekday,Dia da semana,qualitativa,nominal
7,workingday,Se o dia não é fim de semana e nem feriado,qualitativa,nominal
8,weathersit,Clima,qualitativa,nominal
9,temp,Temperatura normalizada em Celsius,quantitativa,contínua


## 2. Preparação de dados:

In [3]:
print("Dados faltantes por coluna:")
print(df.isnull().sum())

Dados faltantes por coluna:
instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64


In [4]:
target_column = 'cnt'

nominal_columns = (
    df_dict
    .query("subtipo == 'nominal'")
    .variavel
    .to_list()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'contínua'")
    .variavel
    .to_list()
)

ordinal_columns = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != 'dteday'")
    .variavel
    .to_list()
)

discrete_columns = (
    df_dict
    .query("subtipo == 'discreta' and variavel != @target_column")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [5]:
if 'dteday' in X.columns:
    X['year'] = X['dteday'].dt.year
    X['month'] = X['dteday'].dt.month
    X['day'] = X['dteday'].dt.day
    X = X.drop(columns=['dteday'])

nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Imputar valores ausentes com a moda
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')),  # Codificação One-Hot
    ('normalization', StandardScaler())  # Normalização
])

continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  # Imputar valores ausentes com a média
    ('normalization', StandardScaler())  # Normalização
])

ordinal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Imputar valores ausentes com a moda
    ('encoding', OrdinalEncoder()),  # Codificação ordinal
    ('normalization', StandardScaler())  # Normalização
])

discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  # Imputar valores ausentes com a média
    ('normalization', StandardScaler())  # Normalização
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('ordinal', ordinal_preprocessor, ordinal_columns),
    ('discrete', discrete_preprocessor, discrete_columns),
    ('continuous', continuous_preprocessor, continuous_columns)
])

In [7]:
X_prepared = preprocessor.fit_transform(X)
print("Dados pré-processados:")
print(X_prepared)

Dados pré-processados:
[[-0.171981   -0.40955052 -0.40727045 ... -0.67994602  1.25017133
  -0.38789169]
 [-0.171981   -0.40955052 -0.40727045 ... -0.74065231  0.47911298
   0.74960172]
 [-0.171981    2.4417012  -0.40727045 ... -1.749767   -1.33927398
   0.74663186]
 ...
 [-0.171981   -0.40955052 -0.40727045 ... -1.42434419  0.87839173
  -0.85355213]
 [-0.171981   -0.40955052 -0.40727045 ... -1.49004895 -1.01566357
   2.06944426]
 [-0.171981    2.4417012  -0.40727045 ... -1.54048197 -0.35406086
  -0.46020122]]
